In [11]:
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import recall_score

## Defining the bucket 
s3 = boto3.resource('s3')
bucket_name = 'data-445'
bucket = s3.Bucket(bucket_name)

## Defining the csv file 
file_key = 'Fall_2021/In_Class_Assignments/framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [12]:
## Defining input and target variables
X = heart[['age', 'totChol', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

## Defining list to store results
md1_recalls = list()
md2_recalls = list()
md3_recalls = list()
md4_recalls = list()

scaler = MinMaxScaler()

for i in range(0, 100):
    print(i)
    ## Splitting the data 
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)
    
    ## 0-1 transformation of inputs
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    #############
    ## Model 1 ##
    #############
    
    md1 = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)
    pred1 = md1.predict_proba(X_test)[:, 1]
    pred1 = np.where(pred1 < 0.1, 0, 1)
    md1_recalls.append(recall_score(Y_test, pred1))
       
    #############
    ## Model 2 ##
    #############
    
    md2 = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)
    pred2 = md2.predict_proba(X_test)[:, 1]
    pred2 = np.where(pred2 < 0.1, 0, 1)
    md2_recalls.append(recall_score(Y_test, pred2))
    
    #############
    ## Model 3 ##
    #############
    
    md3 = AdaBoostClassifier(base_estimator = SVC(kernel = 'rbf', probability = True), n_estimators = 20).fit(X_train, Y_train)
    pred3 = md3.predict_proba(X_test)[:, 1]
    pred3 = np.where(pred3 < 0.1, 0, 1)
    md3_recalls.append(recall_score(Y_test, pred3))
    
    
    #############
    ## Model 4 ##
    #############
    
    md4 = AdaBoostClassifier(base_estimator = SVC(kernel = 'poly', probability = True), n_estimators = 20).fit(X_train, Y_train)
    pred4 = md4.predict_proba(X_test)[:, 1]
    pred4 = np.where(pred4 < 0.1, 0, 1)
    md4_recalls.append(recall_score(Y_test, pred4))    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [14]:
np.mean(md1_recalls)

0.9108035714285714

In [15]:
np.mean(md2_recalls)

1.0

In [16]:
np.mean(md3_recalls)

0.98

In [17]:
np.mean(md4_recalls)

0.9697321428571429